In [1]:
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
import requests
import re
import matplotlib.colors
import numpy as np
from math import ceil
import os
from tqdm import tqdm
from IPython.display import HTML
import plotly.express as px
from matplotlib import animation, rc
rc('animation', html='jshtml')

import warnings
warnings.filterwarnings("ignore")

In [18]:
MYDIR = ("images")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")

images folder already exists.


In [2]:
!pip install openpyxl

In [3]:
!pip install raceplotly
from raceplotly.plots import barplot

In [4]:
# Original Wikipedia Data
df = pd.DataFrame()
html = requests.get('https://en.m.wikipedia.org/wiki/List_of_Indian_states_and_union_territories_by_GDP_per_capita').text
df = pd.read_html(html)

In [5]:
# Additional Data scrapped from microtrends
add_data = pd.read_excel('dataset.xlsx',engine='openpyxl').drop('Unnamed: 0',axis = 'columns')
add_data.drop([0,1],inplace = True)
add_data = add_data.rename(columns = add_data.iloc[0]).reset_index(drop = True)
add_data.drop(0,inplace = True)
add_data = add_data.T.reset_index()
add_data = add_data.rename(columns = add_data.iloc[0]).reset_index(drop = True)
add_data.drop(0,inplace = True)
add_data.drop('1',axis = 'columns',inplace = True)
for col in add_data.columns:
    try:
        add_data[col] = add_data[col].astype(float).round().astype(int)
    except:
        pass

In [6]:
# Preprocessing and merging of original dataframes
df0 = df[0]
df1 = df[1]
df2 = df[2]

df0.drop(['Rank','2019-2020 (USD)'],axis = 'columns',inplace = True)
df1.drop('Rank',axis = 'columns',inplace = True)
df2.drop('Rank',axis = 'columns',inplace = True)
df0 = df0.rename(columns = {'State/Union Territory':'State/Union territory'})

df1.loc[df1['State/Union territory']=='Maharastra','State/Union territory'] = 'Maharashtra'

dfs = pd.merge(df1,df0,on = 'State/Union territory')
dfs = pd.merge(dfs,df2,on = 'State/Union territory')

In [7]:
# State Geometry Data
shp_gdf = gpd.read_file('./India/Indian_states.shp')

In [8]:
# Preprocessing geometry data
shp_gdf.loc[shp_gdf['st_nm']=='Andaman & Nicobar Island','st_nm'] = 'Andaman and Nicobar Islands'
shp_gdf.loc[shp_gdf['st_nm']=='NCT of Delhi','st_nm'] ='Delhi'
shp_gdf.loc[shp_gdf['st_nm']=='Jammu & Kashmir','st_nm'] = 'Jammu and Kashmir'
shp_gdf.loc[shp_gdf['st_nm']=='Arunanchal Pradesh','st_nm'] = 'Arunachal Pradesh'

shp_gdf.drop(shp_gdf.loc[shp_gdf['st_nm']=='Daman & Diu'].index,inplace = True)
shp_gdf.drop(shp_gdf.loc[shp_gdf['st_nm']=='Dadara & Nagar Havelli'].index,inplace = True)
shp_gdf.drop(shp_gdf.loc[shp_gdf['st_nm']=='Lakshadweep'].index,inplace = True)

In [9]:
# Preprocessing additional and merged original data
dfs = dfs.rename(columns = {'State/Union territory':'st_nm'})
add_data = add_data.rename(columns = {'State / Union Territory':'st_nm'})

add_data.loc[add_data['st_nm']=='Andaman & Nicobar Island','st_nm'] = 'Andaman and Nicobar Islands'
add_data.loc[add_data['st_nm']=='Chhattis- garh','st_nm'] ='Chhattisgarh'
add_data.loc[add_data['st_nm']=='Jammu & Kashmir','st_nm'] = 'Jammu and Kashmir'
add_data.loc[add_data['st_nm']=='Jhar- khand','st_nm'] = 'Jharkhand'
add_data.loc[add_data['st_nm']=='Maha- rashtra','st_nm'] = 'Maharashtra'
add_data.loc[add_data['st_nm']=='Megha- laya','st_nm'] = 'Meghalaya'
add_data.loc[add_data['st_nm']=='Telan-gana','st_nm'] = 'Telangana'
add_data.loc[add_data['st_nm']=='Uttara- khand','st_nm'] = 'Uttarakhand'
add_data.loc[add_data['st_nm']=='Chandi- garh','st_nm'] = 'Chandigarh'
add_data.loc[add_data['st_nm']=='Pudu- cherry','st_nm'] = 'Puducherry'
add_data_INI = add_data.loc[add_data['st_nm']=='All-India per capita NNI']
add_data.drop(add_data.loc[add_data['st_nm']=='All-India per capita NNI'].index,inplace = True)

cols = ['st_nm','1997-98   ','1998-99   ','1999-00   ','2001-02   ','2002-03   ',
        '2003-04   ','2004-05   ','2006-07   ','2007-08   ','2008-09   ','2009-10   ']
dfs = pd.merge(dfs,add_data[cols].astype(str),on = 'st_nm')
dfs = dfs.rename(columns = {'1997-98   ':'199798','1998-99   ':'199899','1999-00   ':'199900','2001-02   ':'200102',
                      '2002-03   ':'200203','2003-04   ':'200304','2004-05   ':'200405','2006-07   ':'200607'
                      ,'2007-08   ':'200708','2008-09   ':'200809','2009-10   ':'200910'} )

In [10]:
# Preprocessing final merged dataframe
merged = pd.merge(shp_gdf,dfs,on = 'st_nm')

merged.drop(merged.loc[merged['st_nm']=='Andaman and Nicobar Islands'].index,inplace = True)
merged.drop(merged.loc[merged['st_nm']=='Chandigarh'].index,inplace = True)
merged.drop(merged.loc[merged['st_nm']=='Puducherry'].index,inplace = True)
merged.drop(merged.loc[merged['st_nm']=='Delhi'].index,inplace = True)
merged = merged.set_index('st_nm')

merged = merged.replace('.',np.NaN)
cols = ['199798','199899','199900','200102','200203','200304']
for c in cols:
    merged.loc[merged.index=='Telangana',c] = merged.loc[merged.index=='Andhra Pradesh',c].item()
    
def clean(text):
    text = re.sub('[^0-9]', '', text)
    return text

for col in merged.columns:
    if col not in ['geometry']:
        merged[col] = merged[col].apply(lambda x: clean(x)).astype('int')

In [11]:
def plot_map(title,column,name,cmap,fontsize,fontweight):
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.axis('off')
    ax.set_title(title,fontdict={'fontsize': fontsize, 'fontweight' : fontweight})
    merged.plot(column = column, cmap=cmap, linewidth=0.5, ax=ax, edgecolor='0.2',legend=True,vmin = 10816,vmax=458304,)
    legend = ax.legend()
    plt.savefig(name)
    plt.close(1)

In [12]:
# Defining color map for choropleth
norm = matplotlib.colors.Normalize(-1,1)
colors = [[norm(-1.0), "white"],
          [norm(-0.7), "yellow"],
          [norm(-0.3), "orange"],
          [norm(0), "red"],
            [norm(0.5), "brown"],
          [norm( 1.0), "black"]]

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
fontsize = '15'
fontweight = '3'

In [13]:
plot_map('NSDP Per Capita (Nominal)1996–97','NSDP Per Capita (Nominal)1996–97[3]','./images/00.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)1997–98','199798','./images/01.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)1998–99','199899','./images/02.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)1999–00','199900','./images/03.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2000–01','NSDP Per Capita (Nominal)2000–01[3]','./images/04.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2001–02','200102','./images/05.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2002–03','200203','./images/06.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2003–04','200304','./images/07.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2004–05','200405','./images/08.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2005–06','NSDP Per Capita (Nominal)2005–06[3]','./images/09.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2006–07','200607','./images/10.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2007–08','200708','./images/11.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2008–09','200809','./images/12.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2008–09','200910','./images/13.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2010–11','NSDP Per Capita (Nominal)2010–11[3]','./images/14.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2011–12','NSDP Per Capita (Nominal)2011–12[2][3]','./images/15.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2012–13','NSDP Per Capita (Nominal)2012–13[2]','./images/16.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2013–14','NSDP Per Capita (Nominal)2013–14[2]','./images/17.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2014–15','NSDP Per Capita (Nominal)2014–15[2]','./images/18.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2015–16','NSDP Per Capita (Nominal)2015–16[2]','./images/19.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2016–17','NSDP Per Capita (Nominal)2016–17[2]','./images/20.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2017–18','NSDP Per Capita (Nominal)2017–18[2]','./images/21.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2018–19','NSDP Per Capita (Nominal)2018–19[2]','./images/22.jpg',cmap,'15','3')

plot_map('NSDP Per Capita (Nominal)2019–20','2019-20 (INR)','./images/23.jpg',cmap,'15','3')

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that 

In [14]:
import cv2

In [15]:
def create_animation(ims):
    fig=plt.figure(figsize=(12,12))
    plt.axis('off')
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

imgdir1 ='./images/' 
paths0=[]
for dirname, _, filenames in os.walk(imgdir1):
    for filename in filenames:
        if filename not in ['__notebook_source__.ipynb','__notebook__.ipynb']:
            print(filename)
            paths0+=[os.path.join(dirname, filename)]     
paths0 = sorted(paths0)
images0=[]
for i in tqdm(range(0,len(paths0))):
    images0+=[cv2.imread(paths0[i])]

00.jpg
01.jpg
02.jpg
03.jpg
04.jpg
05.jpg
06.jpg
07.jpg
08.jpg
09.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
20.jpg
21.jpg
22.jpg
23.jpg


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 52.73it/s]


In [16]:
anim = create_animation(np.array(images0))
plt.close(1)

In [17]:
HTML(anim.to_html5_video())